In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

import mlflow
import mlflow.spark


In [2]:
spark = (SparkSession.builder
         .appName("SECOP_Regression")
         .master("spark://spark-master:7077")
         .config("spark.executor.memory", "1g")
         .config("spark.executor.cores", "1")
         .getOrCreate())

spark.sparkContext.setLogLevel("WARN")
print("Spark Version:", spark.version)
print("Spark Master:", spark.sparkContext.master)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/15 02:27:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Version: 3.5.0
Spark Master: spark://spark-master:7077


In [3]:
features_path = "/opt/spark-data/processed/secop_features_q4_2025.parquet"

print("Leyendo:", features_path)

df = spark.read.parquet(features_path)

print("Registros:", df.count())
print("Columnas:", len(df.columns))

df.select("label", "features").show(5, truncate=False)


Leyendo: /opt/spark-data/processed/secop_features_q4_2025.parquet


Registros: 59125
Columnas: 2
+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|label             |features                                                                                                                                             |
+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|16.334818714984902|(75,[0,1,2,3,13,38,57,63],[0.0,0.0,0.1469782368040376,0.029582029790720565,5.951835224792248,2.561845494875754,4.595831606930893,2.0983564491217614])|
|16.797819844969908|(75,[0,1,2,3,8,38,55,64],[0.0,0.0,0.1469782368040376,0.02702555808041138,4.870836736742472,2.561845494875754,2.17106434199968,2.4018714178938714])   |
|17.191595272992377|(75,[0,1,2,3,5,38,56,63],[0.0,0.0,0.1469782368040376,0.1106587040319547,3.5128413176548454,2.561

26/02/15 02:28:11 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
DEV_N = 10000  # podemos subir un poco más ahora
df_dev = df.limit(DEV_N)

print("DEV registros:", df_dev.count())


DEV registros: 10000


In [5]:
train_df, test_df = df_dev.randomSplit([0.8, 0.2], seed=42)

print("Train:", train_df.count())
print("Test:", test_df.count())


Train: 8079
Test: 1921


In [6]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

print("Entrenando modelo de Regresión Lineal (DEV)...")

# Definir modelo
lr = LinearRegression(
    featuresCol="features",
    labelCol="label",
    predictionCol="prediction",
    maxIter=50,
    regParam=0.1,       # regularización (evita sobreajuste)
    elasticNetParam=0.0 # 0 = Ridge, 1 = Lasso
)

# Entrenar modelo
lr_model = lr.fit(train_df)

print("Modelo entrenado correctamente ✅")


Entrenando modelo de Regresión Lineal (DEV)...


26/02/15 02:33:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/02/15 02:33:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
26/02/15 02:33:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Modelo entrenado correctamente ✅


In [7]:
from pyspark.ml.evaluation import RegressionEvaluator

# Predecir en test
pred = lr_model.transform(test_df)
pred.select("label", "prediction").show(10, truncate=False)

# Métricas
rmse_eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
mae_eval  = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")
r2_eval   = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

rmse = rmse_eval.evaluate(pred)
mae  = mae_eval.evaluate(pred)
r2   = r2_eval.evaluate(pred)

print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")
print(f"R2  : {r2:.4f}")


+------------------+-------------------+
|label             |prediction         |
+------------------+-------------------+
|0.0               |17.307977864928592 |
|0.0               |19.700482506679045 |
|0.0               |0.9124897373045435 |
|0.0               |13.830508483426225 |
|0.0               |-1.0660381680002047|
|0.0               |14.15330545172078  |
|0.0               |15.755223527521652 |
|0.0               |17.03588902517645  |
|0.0               |15.335752821705015 |
|0.6931471805599453|14.182845195800107 |
+------------------+-------------------+
only showing top 10 rows



RMSE: 1.7142
MAE : 0.8532
R2  : 0.2440


In [8]:
from pyspark.ml.regression import LinearRegression

lr2 = LinearRegression(
    featuresCol="features",
    labelCol="label",
    predictionCol="prediction",
    maxIter=100,
    regParam=0.01,
    elasticNetParam=0.5
)

lr2_model = lr2.fit(train_df)

pred2 = lr2_model.transform(test_df)

r2_2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2").evaluate(pred2)
print("Nuevo R2:", r2_2)


Nuevo R2: 0.19552595202386502


In [9]:
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluadores
evaluator_rmse = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="rmse"
)

evaluator_mae = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="mae"
)

evaluator_r2 = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="r2"
)

# Métricas
rmse_2 = evaluator_rmse.evaluate(pred2)
mae_2 = evaluator_mae.evaluate(pred2)
r2_2 = evaluator_r2.evaluate(pred2)

print("=== MÉTRICAS MODELO 2 (ElasticNet) ===")
print(f"RMSE: {rmse_2:.4f}")
print(f"MAE : {mae_2:.4f}")
print(f"R2  : {r2_2:.4f}")

# Mostrar algunas predicciones
pred2.select("label", "prediction").show(10, truncate=False)


=== MÉTRICAS MODELO 2 (ElasticNet) ===
RMSE: 1.7078
MAE : 0.7328
R2  : 0.2446
+------------------+------------------+
|label             |prediction        |
+------------------+------------------+
|0.0               |17.41704399343791 |
|0.0               |19.95362626809439 |
|0.0               |0.2620865767414884|
|0.0               |13.849034453320344|
|0.0               |-1.717297111040729|
|0.0               |14.191792641621435|
|0.0               |15.855199435793345|
|0.0               |17.114777899550194|
|0.0               |15.456553118275894|
|0.6931471805599453|14.24322660551498 |
+------------------+------------------+
only showing top 10 rows



In [10]:
print("Entrenando modelo FINAL (FULL)...")

train_full, test_full = df.randomSplit([0.8, 0.2], seed=42)

lr_final = LinearRegression(
    featuresCol="features",
    labelCol="label",
    predictionCol="prediction",
    maxIter=100,
    regParam=0.01,
    elasticNetParam=0.5
)

lr_final_model = lr_final.fit(train_full)

pred_full = lr_final_model.transform(test_full)

r2_full = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2").evaluate(pred_full)

print("R2 FULL:", r2_full)


Entrenando modelo FINAL (FULL)...
R2 FULL: 0.33908001080560213


26/02/15 03:05:26 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
26/02/15 03:05:28 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

# Análisis del Modelo de Regresión Lineal – SECOP Q4 2025

En esta fase se implementó un modelo de regresión lineal para predecir el logaritmo del valor del contrato. Se utilizó una estrategia de división 80/20 para separar los datos en entrenamiento y prueba, garantizando reproducibilidad mediante una semilla fija. Siguiendo la recomendación metodológica del profesor, primero se trabajó con un subconjunto de 10.000 registros (fase DEV) para validar estabilidad y tiempos de ejecución antes de escalar al dataset completo.

En la fase DEV (10.000 registros), el modelo con ElasticNet obtuvo las siguientes métricas sobre el conjunto de prueba:
- **RMSE:** 1.7078  
- **MAE:** 0.7328  
- **R²:** 0.2446  

Esto indica que el modelo logra explicar aproximadamente el **24.46% de la variabilidad** del logaritmo del valor del contrato en datos no vistos durante el entrenamiento. El error absoluto medio de 0.73 en escala logarítmica sugiere que, en promedio, la predicción se desvía menos de una unidad logarítmica respecto al valor real.

Posteriormente, al entrenar el modelo con el dataset completo (59.125 registros), el desempeño mejoró, alcanzando un **R² FULL de 0.3391**, lo que implica que el modelo explica aproximadamente el **33.91% de la variabilidad** total. Este incremento es coherente con el aumento en tamaño muestral, ya que más información permite estimar con mayor precisión los parámetros del modelo.

Aunque el R² no es alto, el comportamiento es consistente con la naturaleza del problema, dado que el valor de los contratos públicos depende de múltiples factores estructurales, administrativos y contextuales que no necesariamente están capturados en las variables disponibles. No se evidencia un sobreajuste severo, ya que el desempeño entre DEV y FULL mantiene coherencia y estabilidad.

En conclusión, la regresión lineal regularizada constituye una línea base sólida. El pipeline es estable, reproducible y escalable, cumpliendo con los requerimientos técnicos del laboratorio. Sin embargo, el nivel de explicación alcanzado sugiere que podrían explorarse modelos no lineales en fases posteriores para capturar relaciones más complejas entre variables.
